## Volviendo a la máquina expendedora

Vamos a trabajar con un modelo para los items de nuestra máquina expendedora, que nos han llegado como [archivo](../data/FoodAndDrinks.csv). El modelo es el siguiente:

In [23]:
[<Measure>]
type oz 

type FoodType = 
    FoodType of string 

type FoodProduct =
    | Eat of FoodType 
    | Drink of FoodType 

type Brand = 
    | Brand of string 
 

type FoodMachineItem =
    {
        Brand: Brand
        ProductType: FoodProduct 
        Price: float 
        Weight: float<oz> 
    }

Algunas funciones de conveniencia:

In [24]:
let prettyPrint item = 
    let (Brand nameBrand) = item.Brand

    let foodType = 
        match item.ProductType with 
        | Eat (FoodType t) -> t 
        | Drink (FoodType t) -> t

    printfn $"{foodType} ({nameBrand}, {item.Weight} oz) costs {item.Price}$"

In [25]:
let readFile(fileName: string) =  
    let lines = File.ReadAllLines(fileName)
    lines 
    

Ahora podemos leer los datos:

In [26]:
let foodAndDrinkData = "../data/FoodAndDrinks.csv"

let products = readFile(foodAndDrinkData)


In [5]:
products.GetType()

System.String[]

In [27]:
printfn "%A" products[0]
printfn "%A" products[1]

"Food,Brand,Weight/Size,Price,Drinks,Brand,Weight/Size,Price"
"Chips,Lay's,1 oz,$1.00,Bottled Water,Aquafina,16 oz,$1.50"


In [28]:
products[0..20]
|> Seq.iteri (fun i s ->  printfn $"{i}: {s}")

0: Food,Brand,Weight/Size,Price,Drinks,Brand,Weight/Size,Price
1: Chips,Lay's,1 oz,$1.00,Bottled Water,Aquafina,16 oz,$1.50
2: Chips,Doritos,2 oz,$2.00,Bottled Water,Dasani,20 oz,$2.00
3: Candy Bars,Snickers,1.5 oz,$1.25,Soda,Coca-Cola,12 oz,$1.50
4: Candy Bars,Kit Kat,1.5 oz,$1.25,Soda,Pepsi,20 oz,$2.00
5: Granola Bars,Nature Valley,1.5 oz,$1.50,Sports Drinks,Gatorade,20 oz,$2.50
6: Cookies,Chips Ahoy,1.5 oz,$1.25,Juice Boxes,Capri Sun,6 oz,$1.00
7: Crackers,Cheez-Its,2 oz,$1.50,Energy Drinks,Red Bull,8.4 oz,$3.00
8: Nuts,Planters,1 oz,$1.50,Iced Tea,Lipton,16 oz,$1.50
9: Popcorn,Pop-Secret,2 oz,$2.00,Lemonade,Minute Maid,16 oz,$1.50
10: Beef Jerky,Jack Link's,1 oz,$3.50,Iced Coffee,Starbucks,12 oz,$2.50
11: Cup Noodles,Maruchan,3 oz,$1.50,Chocolate,Swiss Miss,8 oz,$2.00
12: Pretzels,Rold Gold,1 oz,$1.00,Milk,Nesquik,8 oz,$1.00
13: Trail Mix,Planters,1 oz,$1.50,Chocolate Milk,Yoo-hoo,8 oz,$1.00
14: Rice Krispie Treats,Kellogg's,1.5 oz,$1.25,Fruit Smoothies,Naked Juice,16 oz,$4.00
15: 

Qué tenemos que hacer? Veamos un poco cómo nos llega el dato:

In [8]:
let items =   products[2].Split(',')
printfn "%A" items

[|"Chips"; "Doritos"; "2 oz"; "$2.00"; "Bottled Water"; "Dasani"; "20 oz";
  "$2.00"|]


Esto es una secuencia (`seq`) que tiene dos items. Podemos hacer 

- una función que dado este string nos devuelva dos datos `FoodMachineItem`
- una función que transforme el peso en un tipo `float<oz>`
- una función que transforme el precio en un tipo `float`

In [9]:
let price (s:string) =
    s.Split('$')[1] |> float
    
    
price "$2.00"

2

In [29]:
let weight (s:string) =
    s.Split('o')[0] 
    |> float
    |> (fun v -> v*1.0<oz>)

printfn $"""{weight "20 oz"}"""

20


In [31]:
let createItems (s: string) = 
    let itemList = 
        s.Split(',')
        |> Seq.toList 

    printfn "%A" itemList 
    let eat = 
        {
            Brand = Brand (itemList[1])
            ProductType = Eat (FoodType itemList[0]) 
            Price = price itemList[3]
            Weight = weight itemList[2]
        }
    let drink = 
        {
            Brand = Brand (itemList[5])
            ProductType = Drink (FoodType itemList[4]) 
            Price = price itemList[7]
            Weight = weight itemList[6]
        }

    (eat,drink)        
            



In [32]:
let items =   products[2].Split(',')
printfn "%A" items
createItems products[2]

[|"Chips"; "Doritos"; "2 oz"; "$2.00"; "Bottled Water"; "Dasani"; "20 oz";
  "$2.00"|]
["Chips"; "Doritos"; "2 oz"; "$2.00"; "Bottled Water"; "Dasani"; "20 oz";
 "$2.00"]


({ Brand = Brand "Doritos"\n ProductType = Eat (FoodType "Chips")\n Price = 2.0\n Weight = 2.0 }, { Brand = Brand "Dasani"\n ProductType = Drink (FoodType "Bottled Water")\n Price = 2.0\n Weight = 20.0 }) Item1 { Brand = Brand "Doritos"\n ProductType = Eat (FoodType "Chips")\n Price = 2.0\n Weight = 2.0 } Brand Brand "Doritos" Item Doritos ProductType Eat (FoodType "Chips") Item FoodType "Chips" Item Chips Price 2 Weight 2 Item2 { Brand = Brand "Dasani"\n ProductType = Drink (FoodType "Bottled Water")\n Price = 2.0\n Weight = 20.0 } Brand Brand "Dasani" Item Dasani ProductType Drink (FoodType "Bottled Water") Item FoodType "Bottled Water" Item Bottled Water Price 2 Weight 20

In [33]:
products[1..]
|> Seq.map (fun p -> createItems p)

["Chips"; "Lay's"; "1 oz"; "$1.00"; "Bottled Water"; "Aquafina"; "16 oz";
 "$1.50"]
["Chips"; "Doritos"; "2 oz"; "$2.00"; "Bottled Water"; "Dasani"; "20 oz";
 "$2.00"]
["Candy Bars"; "Snickers"; "1.5 oz"; "$1.25"; "Soda"; "Coca-Cola"; "12 oz";
 "$1.50"]
["Candy Bars"; "Kit Kat"; "1.5 oz"; "$1.25"; "Soda"; "Pepsi"; "20 oz"; "$2.00"]
["Granola Bars"; "Nature Valley"; "1.5 oz"; "$1.50"; "Sports Drinks"; "Gatorade";
 "20 oz"; "$2.50"]
["Cookies"; "Chips Ahoy"; "1.5 oz"; "$1.25"; "Juice Boxes"; "Capri Sun"; "6 oz";
 "$1.00"]
["Crackers"; "Cheez-Its"; "2 oz"; "$1.50"; "Energy Drinks"; "Red Bull"; "8.4 oz";
 "$3.00"]
["Nuts"; "Planters"; "1 oz"; "$1.50"; "Iced Tea"; "Lipton"; "16 oz"; "$1.50"]
["Popcorn"; "Pop-Secret"; "2 oz"; "$2.00"; "Lemonade"; "Minute Maid"; "16 oz";
 "$1.50"]
["Beef Jerky"; "Jack Link's"; "1 oz"; "$3.50"; "Iced Coffee"; "Starbucks";
 "12 oz"; "$2.50"]
["Cup Noodles"; "Maruchan"; "3 oz"; "$1.50"; "Chocolate"; "Swiss Miss"; "8 oz";
 "$2.00"]
["Pretzels"; "Rold Gold"; "1 oz

Error: System.Reflection.TargetInvocationException: Exception has been thrown by the target of an invocation.
 ---> System.ArgumentException: The index was outside the range of elements in the list. (Parameter 'n')
   at Microsoft.FSharp.Collections.PrivateListHelpers.nth[a](FSharpList`1 l, Int32 n) in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4093
   at FSI_0061.createItems(String s)
   at FSI_0064.it@2-22.Invoke(String p)
   at Microsoft.FSharp.Collections.Internal.IEnumerator.map@128.DoMoveNext(b& curr) in D:\a\_work\1\s\src\FSharp.Core\seq.fs:line 130
   at Microsoft.FSharp.Collections.Internal.IEnumerator.MapEnumerator`1.System.Collections.IEnumerator.MoveNext() in D:\a\_work\1\s\src\FSharp.Core\seq.fs:line 113
   at Microsoft.DotNet.Interactive.Formatting.EnumerableExtensions.TakeAndCountRemaining[T](IEnumerable`1 source, Int32 count, Boolean forceCountRemainder) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\EnumerableExtensions.cs:line 23
   at Microsoft.

### Tratando con datos que no están

La forma que F# nos ofrece para tratar con datos que no existen es el tipo `option`: 

```fsharp
type Option<'a> =       
   | Some of 'a           // valid value
   | None                 // missing
```

In [13]:
let emptyStringAsOption (s:string) = 

    if (String.IsNullOrEmpty(s)) then 
        None 
    else 
        Some s

printfn "%A" (emptyStringAsOption "hola")
printfn "%A" (emptyStringAsOption "")        

Some "hola"
None


El tipo `option` tiene métodos similares a los del tipo `Result`

In [35]:
let tryParseInt (s:string) = 
    if (s |> Seq.forall Char.IsDigit) then 
        Some (int s)
    else 
        None 

printfn "%A" (tryParseInt "43")

printfn "%A" (tryParseInt "Messi")


Some 43
None


In [36]:
let divisionBy2 s = 
    s
    |> tryParseInt
    |> Option.map (fun n -> n/2)



In [37]:
printfn "%A" (divisionBy2 "43")   
printfn "%A" (divisionBy2 "Messi") 

Some 21
None


Luego de esta introducción, volvamos a mirar nuestros datos. El problema lo tenemos en la última línea:

In [17]:
products[15]

Protein Bars,Quest,2.1 oz,$3.00,,

In [18]:
products[15].Split(',')

[ Protein Bars, Quest, 2.1 oz, $3.00, , ]

In [38]:
let createItemsRobust (s: string) = 
    let itemList = 
        s.Split(',')
        |> Seq.toList 

    // printfn "%A" itemList 
    let eat = 
        {
            Brand = Brand (itemList[1])
            ProductType = Eat (FoodType itemList[0]) 
            Price = price itemList[3]
            Weight = weight itemList[2]
        }        

    let drink = 
        if (String.IsNullOrEmpty(itemList[4])) then 
            None 
        else 
            {
                Brand = Brand (itemList[5])
                ProductType = Drink (FoodType itemList[4]) 
                Price = price itemList[7]
                Weight = weight itemList[6]
            } |> Some 

    (eat,drink)        
            



In [39]:
let mixedSeq = 
    products[1..]
    |> Seq.map (fun p -> createItemsRobust p)

mixedSeq     
|> Seq.iter (fun i -> printfn "%A" i)

({ Brand = Brand "Lay's"
   ProductType = Eat (FoodType "Chips")
   Price = 1.0
   Weight = 1.0 }, Some { Brand = Brand "Aquafina"
                          ProductType = Eat (FoodType "Bottled Water")
                          Price = 1.5
                          Weight = 16.0 })
({ Brand = Brand "Doritos"
   ProductType = Eat (FoodType "Chips")
   Price = 2.0
   Weight = 2.0 }, Some { Brand = Brand "Dasani"
                          ProductType = Eat (FoodType "Bottled Water")
                          Price = 2.0
                          Weight = 20.0 })
({ Brand = Brand "Snickers"
   ProductType = Eat (FoodType "Candy Bars")
   Price = 1.25
   Weight = 1.5 }, Some { Brand = Brand "Coca-Cola"
                          ProductType = Eat (FoodType "Soda")
                          Price = 1.5
                          Weight = 12.0 })
({ Brand = Brand "Kit Kat"
   ProductType = Eat (FoodType "Candy Bars")
   Price = 1.25
   Weight = 1.5 }, Some { Brand = Brand "Pepsi"
              

In [40]:
let s = [ Some 3; Some 2; None; Some 42]

In [41]:
s
|> List.choose id 

[ 3, 2, 42 ] HeadOrDefault 3 TailOrNull [ 2, 42 ] HeadOrDefault 2 TailOrNull [ 42 ] HeadOrDefault 42 TailOrNull [ ] HeadOrDefault 0 TailOrNull <null> Head System.InvalidOperationException: The input list was empty.\n at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4141\n at lambda_method513(Closure, FSharpList`1)\n at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrE... TargetSite T get_Head() Name get_Head DeclaringType Microsoft.FSharp.Collections.FSharpList`1[T] ReflectedType Microsoft.FSharp.Collections.FSharpList`1[T] MemberType Method MetadataToken 100663764 Module FSharp.Core.dll IsSecurityCritical True IsSecuritySafeCritical False IsSecurityTransparent False MethodHandle System.RuntimeMethodHandle Attributes Public, HideBySig, SpecialName CallingConvention Standard, HasThis ReturnType T ReturnTypeCustomAttributes T ReturnParameter T IsCollectible False IsGenericMethod False IsGenericMethodDefinition False ContainsGenericParameters True MethodImplementationFlags IL IsAbstract False IsConstructor False IsFinal False IsHideBySig True IsSpecialName True IsStatic False IsVirtual False IsAssembly False IsFamily False IsFamilyAndAssembly False IsFamilyOrAssembly False IsPrivate False IsPublic True IsConstructedGenericMethod False CustomAttributes [ ] Message The input list was empty. Data (empty) InnerException <null> HelpLink <null> Source FSharp.Core HResult -2146233079 StackTrace at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4141
 at lambda_method513(Closure, FSharpList`1)
 at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T instance) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\MemberAccessor{T}.cs:line 58 Tail System.InvalidOperationException: The input list was empty.\n at Microsoft.FSharp.Collections.FSharpList`1.get_Tail() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4146\n at lambda_method514(Closure, FSharpList`1)\n at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrE... TargetSite Microsoft.FSharp.Collections.FSharpList`1[T] get_Tail() Name get_Tail DeclaringType Microsoft.FSharp.Collections.FSharpList`1[T] ReflectedType Microsoft.FSharp.Collections.FSharpList`1[T] MemberType Method MetadataToken 100663765 Module FSharp.Core.dll IsSecurityCritical True IsSecuritySafeCritical False IsSecurityTransparent False MethodHandle System.RuntimeMethodHandle Attributes Public, HideBySig, SpecialName CallingConvention Standard, HasThis ReturnType Microsoft.FSharp.Collections.FSharpList`1[T] ReturnTypeCustomAttributes Microsoft.FSharp.Collections.FSharpList`1[T] ReturnParameter Microsoft.FSharp.Collections.FSharpList`1[T] IsCollectible False IsGenericMethod False IsGenericMethodDefinition False ContainsGenericParameters True MethodImplementationFlags IL IsAbstract False IsConstructor False IsFinal False IsHideBySig True IsSpecialName True IsStatic False IsVirtual False IsAssembly False IsFamily False IsFamilyAndAssembly False IsFamilyOrAssembly False IsPrivate False IsPublic True IsConstructedGenericMethod False CustomAttributes [ ] Message The input list was empty. Data (empty) InnerException <null> HelpLink <null> Source FSharp.Core HResult -2146233079 StackTrace at Microsoft.FSharp.Collections.FSharpList`1.get_Tail() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4146
 at lambda_method514(Closure, FSharpList`1)
 at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T instance) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\MemberAccessor{T}.cs:line 58 (values) [ ] Head 42 Tail [ ] HeadOrDefault 0 TailOrNull <null> Head System.InvalidOperationException: The input list was empty.\n at Microsoft.FSharp.Collections.FSharpList`1.get_Head() in D:\a\_work\1\s\src\FSharp.Core\prim-types.fs:line 4141\n at lambda_method513(Closure, FSharpList`1)\n at Microsoft.DotNet.Interact

In [42]:
let eats = 
    mixedSeq
    |> Seq.map fst 

let drinks = 
    mixedSeq
    |> Seq.map snd 
    |> Seq.choose id 

let all = 
    seq { 
        yield! eats
        yield! drinks 
    }


In [43]:
all
|> Seq.iter prettyPrint

Chips (Lay's, 1 oz) costs 1$
Chips (Doritos, 2 oz) costs 2$
Candy Bars (Snickers, 1.5 oz) costs 1.25$
Candy Bars (Kit Kat, 1.5 oz) costs 1.25$
Granola Bars (Nature Valley, 1.5 oz) costs 1.5$
Cookies (Chips Ahoy, 1.5 oz) costs 1.25$
Crackers (Cheez-Its, 2 oz) costs 1.5$
Nuts (Planters, 1 oz) costs 1.5$
Popcorn (Pop-Secret, 2 oz) costs 2$
Beef Jerky (Jack Link's, 1 oz) costs 3.5$
Cup Noodles (Maruchan, 3 oz) costs 1.5$
Pretzels (Rold Gold, 1 oz) costs 1$
Trail Mix (Planters, 1 oz) costs 1.5$
Rice Krispie Treats (Kellogg's, 1.5 oz) costs 1.25$
Protein Bars (Quest, 2.1 oz) costs 3$
Bottled Water (Aquafina, 16 oz) costs 1.5$
Bottled Water (Dasani, 20 oz) costs 2$
Soda (Coca-Cola, 12 oz) costs 1.5$
Soda (Pepsi, 20 oz) costs 2$
Sports Drinks (Gatorade, 20 oz) costs 2.5$
Juice Boxes (Capri Sun, 6 oz) costs 1$
Energy Drinks (Red Bull, 8.4 oz) costs 3$
Iced Tea (Lipton, 16 oz) costs 1.5$
Lemonade (Minute Maid, 16 oz) costs 1.5$
Iced Coffee (Starbucks, 12 oz) costs 2.5$
Chocolate (Swiss Miss, 8 o